## generateFinalDataset

*   Input:  
  1.   *clean_columns_everything.csv*  
  2.   *svi_edited.csv* 
*   Does:  
  1.   Data cleaning
  2.   Aggregating incidents over census tract
  3.   Merging census tracts with ACS and SVI data
  4.   Splitting rows into training and test sets
  5.   Feature scaling
  6.   Creating target variables
*   Output:
  1.   *trainTractsDroppedStandard.csv*
  2.   *testTractsDroppedStandard.csv*



In [ ]:
# Install libraries
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


clean_cols = pd.read_csv('data_source/clean_columns_everything.csv', 
                            parse_dates=True,
                            infer_datetime_format=True,
                            engine='python'
                           )

svi_cols = pd.read_csv('data_source/svi_edited.csv', 
                            engine='python'
                           )

# Allow us to see all columns of our dataframe
pd.set_option('max_columns', None)

## Data Cleaning and creating 'week' utility column

In [ ]:
# Remove duplicates
clean_cols['duplicate_identifier'] = clean_cols['group'] + clean_cols['incident_id']
clean_cols.drop_duplicates(subset=['duplicate_identifier'], inplace=True)

In [ ]:
# Replace NaN in NFIRS codes with -999
replace_dict = {'nfirs_group_final': {np.nan:'-999', 'U':'999'}, 
                'nfirs_category_final': {np.nan:'-999', 'UU':'999'}, 
                'nfirs_code_final': {np.nan:'-999', 'UUU':'999'}}
clean_cols.replace(replace_dict, inplace=True)

In [ ]:
# Convert 'occurred_on' dtype from 'object' --> 'datetime'
clean_cols['occurred_on'] = pd.to_datetime(clean_cols['occurred_on'], format='%Y-%m-%d %H:%M:%S')

# Convert NFIRS code column dtypes from 'object' --> 'int16'
clean_cols['nfirs_group_final'] = clean_cols['nfirs_group_final'].astype('int16')
clean_cols['nfirs_category_final'] = clean_cols['nfirs_category_final'].astype('int16')
clean_cols['nfirs_code_final'] = clean_cols['nfirs_code_final'].astype('int16')

# Replace 0 --> NaN so that it can be imputed
clean_cols.replace({'median_year_built': {0.0: np.nan}}, inplace=True)

# Round values to two decimal places
clean_cols = clean_cols.round(2)

In [ ]:
# Create a temporary column containing values denoting week in the year (1-52)
temp_col = clean_cols['occurred_on'].dt.isocalendar().week

# Insert the column after "day" at index position 62
clean_cols.insert(62, 'week', temp_col)

## Aggregating total, medical, and fire incident counts over census tract

In [ ]:
# Create dataframe of total incidents grouped by TRACT
groupByTract = clean_cols.groupby(['TRACTA']).size().reset_index(name='total_incidents')

# Add column of total medical incidents grouped by TRACT
medicalGroupByTract = clean_cols[clean_cols['nfirs_group_final'] == 3].groupby(['TRACTA']).size().reset_index(name='medical_incidents')
groupByTract = groupByTract.merge(medicalGroupByTract, on='TRACTA', how='left')

# Add column of total fire incidents grouped by TRACT
fireGroupByTract = clean_cols[clean_cols['nfirs_group_final'] == 1].groupby(['TRACTA']).size().reset_index(name='fire_incidents')
groupByTract = groupByTract.merge(fireGroupByTract, on='TRACTA', how='left')

## Finding the number of days a census tract has "been in the system"¶

In [ ]:
# Figuring out when was earliest report by tract
earliestTractIncident = clean_cols.groupby('TRACTA')['occurred_on'].agg(['min'], on='occurred_on').rename({'min':'earliest_incident'},axis=1).reset_index()

# Figuring out when was latest report by tract
latestTractIncident = clean_cols.groupby('TRACTA')['occurred_on'].agg(['max'], on='occurred_on').rename({'max':'latest_incident'},axis=1).reset_index()

# Add columns showing when census tract's first and last incident appeared in our dataset
groupByTract = groupByTract.merge(earliestTractIncident, on='TRACTA', how='left')
groupByTract = groupByTract.merge(latestTractIncident, on='TRACTA', how='left')

# Convert columns to datetime
groupByTract['earliest_incident'] = pd.to_datetime(groupByTract['earliest_incident'], format='%Y-%m-%d %H:%M:%S')
groupByTract['latest_incident'] = pd.to_datetime(groupByTract['latest_incident'], format='%Y-%m-%d %H:%M:%S')

# Add column for the number of days that elapsed between 'earliest_incident' and 'latest_incident'
groupByTract['diffTractFirstLastInc'] = (groupByTract['latest_incident'] - groupByTract['earliest_incident']).dt.days

# Replace '0' with '1' in diffTractFirstLastInc
groupByTract.replace(to_replace={'diffTractFirstLastInc': {0: 1}}, inplace=True)

## Add indicator variables flagging mis-reported census tracts¶

In [ ]:
# Create helper column that will help us identify mis-reported census tracts
groupByTract['logTotalIncidents'] = np.log(groupByTract['total_incidents'])

In [ ]:
# Create indicator variable flagging first mis-represented group of census tracts
groupByTract['firstTroubleGroup'] = ((groupByTract['diffTractFirstLastInc'] > 425) & 
                                    (groupByTract['diffTractFirstLastInc'] < 475) & 
                                    (groupByTract['logTotalIncidents'] > 3.5)).astype(int)

# Create indicator variable flagging second mis-represented group of census tracts
groupByTract['secondTroubleGroup'] = ((groupByTract['diffTractFirstLastInc'] > 700) &
                                     (groupByTract['diffTractFirstLastInc'] < 750) &
                                     (groupByTract['logTotalIncidents'] > 3.5)).astype(int)

# Create indicator variable flagging third mis-represented group of census tracts
groupByTract['thirdTroubleGroup'] = ((groupByTract['diffTractFirstLastInc'] > 1000) &
                                    (groupByTract['diffTractFirstLastInc'] < 1100) &
                                    (groupByTract['logTotalIncidents'] > 4.5)).astype(int)

## Collecting demographics to supplement census tract aggregate incident data

In [ ]:
# Columns to drop that don't represent a census tract
drop_cols = ['Unnamed: 0','id', 'nfirs_group_final', 'incident_id', 'alternate_id', 
            'duplicate_identifier', 'nfirs_category_final', 'nfirs_code_final',
            'occurred_on', 'week', 'day', 'month', 'year', 'hour_of_day', 
            'day_of_week', 'day_of_month', 'day_of_year', 'month_of_year',
            'zip_code_lookup']

# Get unique census tracts from incident data and drop columns relevant to incident but not to census tract
uniqueCensusTracts = clean_cols.drop_duplicates(subset=['TRACTA'], inplace=False)
uniqueCensusTracts = uniqueCensusTracts.drop(columns=drop_cols)

# Generate subsets of uniqueCensusTracts so that we can rearrange the dataframe in a more intutitive way
censusCoreInfo = uniqueCensusTracts.loc[:, 'group':'TRACTA']
censusPopStats = uniqueCensusTracts.loc[:, 'total_pop':'percent_2.01ormore_per_room']
censusPopStats2 = uniqueCensusTracts.loc[:, 'median_age':'percent_over90min_commute']

# Rearrange the dataframe
uniqueTractsRearranged = pd.concat([censusCoreInfo, censusPopStats, censusPopStats2], axis=1)

## Create base dataset of census tracts, their demographics, and their aggregated incidents

In [ ]:
########### Merge (census tract aggregate incident data) with (demographic data) ###########


# Merge dataframes to collect the info of unique census tracts (from both), 
# aggregate incident counts (from 'groupByTract'), and demographic data (from 'uniqueTractsRearranged')
censusTractDemographics = groupByTract.merge(uniqueTractsRearranged, on='TRACTA', how='left')


########### Remove census tracts that don't contain any demographic data ###########

# Get census tracts that should be removed (contain many NaN or demographics that are nonsensical)
dropSet1 = set(censusTractDemographics[censusTractDemographics.loc[:,'percent_male':'percent_family'].isnull().all(axis=1)].TRACTA.tolist())
dropSet2 = set(censusTractDemographics[censusTractDemographics.loc[:,'percent_under_0.5_IPL':'median_household_income'].isnull().all(axis=1)].TRACTA.tolist())
dropSetMerged = dropSet1.union(dropSet2)
dropSetMerged.add(12300)

# Update dataset to remove these census tracts
censusTractDemographics = censusTractDemographics[~censusTractDemographics['TRACTA'].isin(dropSetMerged)]

## Create target variables

In [ ]:
# Create target variables normalized by population
censusTractDemographics['incidents_per_capita'] = censusTractDemographics['total_incidents'] / censusTractDemographics['total_pop']
censusTractDemographics['medical_per_capita'] = censusTractDemographics['medical_incidents'] / censusTractDemographics['total_pop']
censusTractDemographics['fire_per_capita'] = censusTractDemographics['fire_incidents'] / censusTractDemographics['total_pop']

## Join SVI Data

In [ ]:
# Join data on 'GISJOIN'
censusTractDemographics = censusTractDemographics.merge(svi_cols, on='GISJOIN', how='left')

## Remove utility / useless columns¶

In [ ]:
# Specify columns to remove
remove_cols = ['medical_incidents', 'fire_incidents', 
               'earliest_incident', 'latest_incident', 'diffTractFirstLastInc',
               'group', 'agency', 'tract_block_group_centroid_lat', 'tract_block_group_centroid_lng',
               'GISJOIN', 'COUNTY', 'Unnamed: 0', 'logTotalIncidents', 'percent_1_person_hh']

# Drop the columns
censusTractDemographics.drop(columns=remove_cols, inplace=True)

## Drop "Trouble" Tracts

In [ ]:
# Get "Trouble" Tracts for analysis
droppedTroubleTracts = censusTractDemographics[(censusTractDemographics.firstTroubleGroup == 1) | 
                                                  (censusTractDemographics.secondTroubleGroup == 1) |
                                                  (censusTractDemographics.thirdTroubleGroup == 1)]

# Drop "Trouble" Tracts
censusTractDemographics = censusTractDemographics[(censusTractDemographics.firstTroubleGroup == 0) & 
                                                  (censusTractDemographics.secondTroubleGroup == 0) &
                                                  (censusTractDemographics.thirdTroubleGroup == 0)]


censusTractDemographics.drop(columns=['firstTroubleGroup',
                                      'secondTroubleGroup',
                                      'thirdTroubleGroup'] , inplace=True)

## Split the Data

In [ ]:
# 80-20 Train-Test Split
trainTracts, testTracts = train_test_split(censusTractDemographics, 
                                           train_size=0.8, test_size=0.2, random_state=1)

## Impute Missing Values

In [ ]:
# Get the medians of the train and test sets
trainMedianMedianYearBuilt = trainTracts['median_year_built'].median()
testMedianMedianYearBuilt = testTracts['median_year_built'].median()

# Impute missing values in 'median_year_built' 
trainTracts.replace({'median_year_built': {np.nan: trainMedianMedianYearBuilt}}, inplace=True)
testTracts.replace({'median_year_built': {np.nan: testMedianMedianYearBuilt}}, inplace=True)

## One-Hot Encode the 'STATE' Column

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# Apply one-hot encoder to the relevant columns
OH_encoder = LabelBinarizer()
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(trainTracts['STATE']))
OH_cols_test = pd.DataFrame(OH_encoder.transform(testTracts['STATE']))

# Replace default column names with more descriptive ones
OH_cols_train.columns = OH_encoder.classes_
OH_cols_test.columns = OH_encoder.classes_

# One-hot encoding removed index; put it back
OH_cols_train.index = trainTracts.index
OH_cols_test.index = testTracts.index

# Remove categorical columns (will replace with one-hot encoding)
trainTracts.drop('STATE', axis=1, inplace=True)
testTracts.drop('STATE', axis=1, inplace=True)

# Add one-hot encoded columns to numerical features
trainTracts = pd.concat([trainTracts, OH_cols_train], axis=1)
testTracts = pd.concat([testTracts, OH_cols_test], axis=1)

## Convert NaN --> 0 for medical_per_capita and fire_per_capita target colums

In [ ]:
# Replace NaN in 0
replace_dict = {'medical_per_capita': {np.nan:0}, 
                'fire_per_capita': {np.nan:0}}

trainTracts.replace(replace_dict, inplace=True)
testTracts.replace(replace_dict, inplace=True)

## Normalize Features

In [ ]:
# Function to generate a kdeplot and Q-Q plot of a given feature
def normalizeHelper(df, feature):  
    # kdeplot on the left
    plt.figure(figsize=(15,4))
    plt.subplot(1, 2, 1)
    plt.title(feature, fontsize=15)
    sns.kdeplot(df[feature], shade=True)

    # Q-Q plot on the right
    plt.subplot(1, 2, 2)
    stats.probplot(df[feature], dist="norm", plot=pylab)

    plt.show()

# # To visualize the distributions of our features
# for feature in trainTracts.loc[:,'total_pop':'SPL_THEMES'].columns:
#     normalizeHelper(trainTracts, feature)

In [ ]:
# Get columns that already exhibit a Gaussian distribution
colAlreadyGaussian = ['percent_male', 'median_year_built',
                      'percent_0to19_years', 'percent_40to64_years', 'percent_employer_only',
                      'percent_2_person_hh', 'percent_3_person_hh', 'percent_4_person_hh', 
                      'percent_0.5to1_per_room', 'percent_2.01ormore_per_room',
                      'median_age', 'percent_leave_for_work_6to9am',
                      'percent_15to24min_commute', 'percent_25to34min_commute', 'SPL_THEME2',
                      'RPL_THEME3', 'SPL_THEME4', 'SPL_THEMES']

colsToNormalize = list(trainTracts.loc[:,'total_pop':'SPL_THEMES'].columns)
colsToNormalize.remove('incidents_per_capita')
colsToNormalize.remove('medical_per_capita')
colsToNormalize.remove('fire_per_capita')

# Get columns that do NOT already exhibit a Guassian distribution
nonGaussianCols = [col for col in colsToNormalize if not col in colAlreadyGaussian]

## SQRT Transform

In [ ]:
colsToSqrt = ['total_pop', 'percent_20to39_years', 'percent_65andover', 'percent_multi_race',
              'percent_hispanic', 'percent_living_alone', 'percent_nonfamily_not_alone',
              'percent_graduate_degree', 'percent_under_0.5_IPL', 'percent_0.5to0.99_IPL',
              'percent_1to1.24_IPL', 'percent_1.25to1.49_IPL', 'percent_1.5to1.84_IPL',
              'percent_1.85to1.99_IPL', 'percent_ue_in_labor_force', 'percent_no_health_insur',
              'percent_direct_purchase_only', 'percent_medicare_only', 'percent_medicaid_only',
              'percent_renter_occ', 'percent_5_person_hh', 'percent_leave_for_work_before_6am',
              'percent_leave_for_work_12to4pm', 'percent_leave_for_work_4tomidnight',
              'percent_5to14min_commute', 'percent_35to44min_commute', 'percent_45to59min_commute',
              'percent_60to89min_commute', 'percent_over90min_commute','EP_LIMENG', 
              'RPL_THEME1', 'RPL_THEME4', 'RPL_THEMES', 'SPL_THEME1']

# Create columns formed by taking the square root
def sqrtTransform(df, feature):
    df[feature + '_' + 'sqrt'] = np.sqrt(df[feature])
    return normalizeHelper(df, feature + '_' + 'sqrt')

# Add SQRT-transformed features to trainTracts
for feature in colsToSqrt:
    sqrtTransform(trainTracts, feature)
    

# Add SQRT-transformed features to testTracts
for feature in colsToSqrt:
    sqrtTransform(testTracts, feature)

## LOG Transform

In [ ]:
colsToLog = ['median_household_income', 'percent_combo_health_insur']

# Create columns formed by taking the natural log
def logTransform(df, feature):
    df[feature + '_' + 'log'] = np.log(df[feature])
    return normalizeHelper(df, feature + '_' + 'log')

# Add LOG-transformed features to trainTracts
for feature in colsToLog:
    logTransform(trainTracts, feature)
    
# Add LOG-transformed features to testTracts
for feature in colsToLog:
    logTransform(testTracts, feature)

## BOXCOX Transform

In [ ]:
colsToBoxCox = ['percent_family', 'percent_bachelors_degree_or_higher', 'percent_over_2_IPL',
                'percent_in_labor_force', 'num_housing_units', 'percent_under0.5_per_room', 
                'percent_drive_to_work']

# Create columns formed by taking the box-cox transform
def boxCoxTransform(df, feature):
    df[feature + '_' + 'boxcox'], param = stats.boxcox(df[feature])
    return normalizeHelper(df, feature + '_' + 'boxcox')

# Add BOXCOX-transformed features to trainTracts
for feature in colsToBoxCox:
    boxCoxTransform(trainTracts, feature)
    
# Add BOXCOX-transformed features to testTracts
for feature in colsToBoxCox:
    boxCoxTransform(testTracts, feature)

## EXPONENTIAL Transform

In [ ]:
colsToExponent = ['percent_white_alone', 'percent_black_alone', 'percent_asian_alone',
                  'percent_other_race_alone', 'percent_HS_diploma_or_higher','percent_some_college_or_higher',
                  'percent_w_public_assist_income', 'percent_vacant', 'percent_tricare_only', 'percent_VA_only',
                  'percent_6_person_hh', 'percent_7ormore_person_hh', 'percent_1.01to1.5_per_room',
                  'percent_1.51to2_per_room', 'percent_walk_or_bike_to_work', 'percent_public_transpo_to_work', 
                  'percent_under5min_commute', 'EP_GROUPQ', 'RPL_THEME2', 'SPL_THEME3']


# Transform the remaining, hard-to-transform columns to a Gaussian distribution by raising to a power
def exponentialTransform(df, feature, powerOf):
    df[feature + '_' + 'exp' + str(powerOf)] = df[feature]**(powerOf)
    return normalizeHelper(df, feature + '_' + 'exp' + str(powerOf))

# Add EXPONENT-transformed features to trainTracts
exponentialTransform(trainTracts, 'percent_white_alone', (1.4))
exponentialTransform(trainTracts, 'percent_black_alone', (0.3))
exponentialTransform(trainTracts, 'percent_asian_alone', (0.3))
exponentialTransform(trainTracts, 'percent_other_race_alone', (0.3))
exponentialTransform(trainTracts, 'percent_HS_diploma_or_higher', (3.6))
exponentialTransform(trainTracts, 'percent_some_college_or_higher', (1.5))
exponentialTransform(trainTracts, 'percent_w_public_assist_income', (0.3))
exponentialTransform(trainTracts, 'percent_vacant', (0.2))
exponentialTransform(trainTracts, 'percent_tricare_only', (0.7))
exponentialTransform(trainTracts, 'percent_VA_only', (0.3))
exponentialTransform(trainTracts, 'percent_6_person_hh', (0.4))
exponentialTransform(trainTracts, 'percent_7ormore_person_hh', (0.4))
exponentialTransform(trainTracts, 'percent_1.01to1.5_per_room', (0.4))
exponentialTransform(trainTracts, 'percent_1.51to2_per_room', (0.3))
exponentialTransform(trainTracts, 'percent_walk_or_bike_to_work', (0.3))
exponentialTransform(trainTracts, 'percent_public_transpo_to_work', (0.4))
exponentialTransform(trainTracts, 'percent_under5min_commute', (0.3))
exponentialTransform(trainTracts, 'EP_GROUPQ', (0.2))
exponentialTransform(trainTracts, 'RPL_THEME2', (0.6))
exponentialTransform(trainTracts, 'SPL_THEME3', (1.2))


# Add EXPONENT-transformed features to testTracts
exponentialTransform(testTracts, 'percent_white_alone', (1.4))
exponentialTransform(testTracts, 'percent_black_alone', (0.3))
exponentialTransform(testTracts, 'percent_asian_alone', (0.3))
exponentialTransform(testTracts, 'percent_other_race_alone', (0.3))
exponentialTransform(testTracts, 'percent_HS_diploma_or_higher', (3.6))
exponentialTransform(testTracts, 'percent_some_college_or_higher', (1.5))
exponentialTransform(testTracts, 'percent_w_public_assist_income', (0.3))
exponentialTransform(testTracts, 'percent_vacant', (0.2))
exponentialTransform(testTracts, 'percent_tricare_only', (0.7))
exponentialTransform(testTracts, 'percent_VA_only', (0.3))
exponentialTransform(testTracts, 'percent_6_person_hh', (0.4))
exponentialTransform(testTracts, 'percent_7ormore_person_hh', (0.4))
exponentialTransform(testTracts, 'percent_1.01to1.5_per_room', (0.4))
exponentialTransform(testTracts, 'percent_1.51to2_per_room', (0.3))
exponentialTransform(testTracts, 'percent_walk_or_bike_to_work', (0.3))
exponentialTransform(testTracts, 'percent_public_transpo_to_work', (0.4))
exponentialTransform(testTracts, 'percent_under5min_commute', (0.3))
exponentialTransform(testTracts, 'EP_GROUPQ', (0.2))
exponentialTransform(testTracts, 'RPL_THEME2', (0.6))
exponentialTransform(testTracts, 'SPL_THEME3', (1.2))

## Move Columns Around (For Convenience / Viewing Purposes)

In [ ]:
# Get the 'parent' (nonGaussianCols) and 'child' (colNewlyGaussian) features
nonGaussianCols = [col for col in colsToNormalize 
                   if not col in colAlreadyGaussian]
colNewlyGaussian = trainTracts.loc[:, 'total_pop_sqrt':'SPL_THEME3_exp1.2'].columns


# Drop non-Gaussian columns from trainTracts and append them to the end of the dataset
trainTractsNonGaussian = trainTracts[nonGaussianCols]
trainTracts.drop(columns=nonGaussianCols, inplace=True)
trainTracts = pd.concat([trainTracts, trainTractsNonGaussian], axis=1)

# Drop non-Gaussian columns from testTracts and append them to the end of the dataset
testTractsNonGaussian = testTracts[nonGaussianCols]
testTracts.drop(columns=nonGaussianCols, inplace=True)
testTracts = pd.concat([testTracts, testTractsNonGaussian], axis=1)

In [ ]:
targetCols = ['incidents_per_capita', 'medical_per_capita', 'fire_per_capita']

# Drop target column from trainTracts and append to the end of the dataset
trainTractsTarget = trainTracts[targetCols]
trainTracts.drop(columns=targetCols, inplace=True)
trainTracts = pd.concat([trainTracts, trainTractsTarget], axis=1)

# Drop target columns from testTracts and append to the end of the dataset
testTractsTarget = testTracts[targetCols]
testTracts.drop(columns=targetCols, inplace=True)
testTracts = pd.concat([testTracts, testTractsTarget], axis=1)

## Output Data to .CSV

In [ ]:
# Output splits of aggregated census tracts to .csv
trainTracts.to_csv('data_source/trainTractsDroppedStandard.csv')
testTracts.to_csv('data_source/trainTractsDroppedStandard.csv')